In [16]:
import pandas as pd
import numpy as np

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [17]:
drivers = pd.read_csv('drivers.csv', header=0)
constructors = pd.read_csv('constructors.csv', header=0)
results = pd.read_csv('results.csv', header=0)
races = pd.read_csv('races.csv', header=0)

In [18]:
len(races)

1079

In [32]:
list(range(1950,2021))

[1950,
 1951,
 1952,
 1953,
 1954,
 1955,
 1956,
 1957,
 1958,
 1959,
 1960,
 1961,
 1962,
 1963,
 1964,
 1965,
 1966,
 1967,
 1968,
 1969,
 1970,
 1971,
 1972,
 1973,
 1974,
 1975,
 1976,
 1977,
 1978,
 1979,
 1980,
 1981,
 1982,
 1983,
 1984,
 1985,
 1986,
 1987,
 1988,
 1989,
 1990,
 1991,
 1992,
 1993,
 1994,
 1995,
 1996,
 1997,
 1998,
 1999,
 2000,
 2001,
 2002,
 2003,
 2004,
 2005,
 2006,
 2007,
 2008,
 2009,
 2010,
 2011,
 2012,
 2013,
 2014,
 2015,
 2016,
 2017,
 2018,
 2019,
 2020]

In [19]:
# Filer raceId here


season = list(range(1950,2022)) #Enter a F1 World Championship year
results = results[
    results['raceId'].isin(
        races[races['year'].isin(season)]['raceId'].values.tolist()
        )
    ]


### Edges

In [20]:
drivers_results = pd.merge(drivers, results, on='driverId')
constructors_results = pd.merge(constructors, results, on='constructorId')
drivers_constructors = pd.merge(drivers_results, constructors_results, on='resultId')
drivers_constructors = drivers_constructors[['driverRef', 'forename', 'surname', "dob", 'nationality_x', 'constructorRef', 'name', 'raceId_x']].drop_duplicates()
drivers_constructors.reset_index(inplace=True, drop=True)

In [21]:
dc_1 = pd.merge(drivers_constructors, drivers_constructors, on=['raceId_x', 'constructorRef'])
dc_1 = dc_1[['driverRef_x', 'driverRef_y']]
dc_1['connection'] = 'constructor'
dc_1.drop_duplicates(inplace=True)
#dc_1

In [22]:
driver_edges = dc_1

In [23]:
edges = driver_edges[driver_edges['driverRef_x'] != driver_edges['driverRef_y']]
edges.reset_index(inplace=True, drop=True)

In [24]:
edges_list = edges.values.tolist()
edges_set = [set(dl) for dl in edges_list] 
repeat_connections = [idx for idx, ds in enumerate(edges_set) if ds in edges_set[:idx]]

In [25]:
edges.drop(repeat_connections, axis=0, inplace=True)
edges.reset_index(inplace=True, drop=True)

C:\Users\varma\AppData\Local\Temp\ipykernel_34728\1200515550.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  edges.drop(repeat_connections, axis=0, inplace=True)


### Nodes

In [26]:
nodes = drivers_constructors[['driverRef', 'forename', 'surname', 'dob', 'nationality_x']]
nodes.drop_duplicates(inplace=True)
nodes.reset_index(drop=True, inplace=True)

C:\Users\varma\AppData\Local\Temp\ipykernel_34728\2592146589.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nodes.drop_duplicates(inplace=True)


### Writing to files

In [27]:
edges.to_csv(edges_fpath, index = False)
nodes.to_csv(nodes_fpath, index = False)

In [28]:
nodes
edges

,driverRef,forename,surname,dob,nationality_x
0,hamilton,Lewis,Hamilton,1985-01-07,British
1,heidfeld,Nick,Heidfeld,1977-05-10,German
2,rosberg,Nico,Rosberg,1985-06-27,German
3,alonso,Fernando,Alonso,1981-07-29,Spanish
4,kovalainen,Heikki,Kovalainen,1981-10-19,Finnish
...,...,...,...,...,...
848,pietro_fittipaldi,Pietro,Fittipaldi,1996-06-25,Brazilian
849,aitken,Jack,Aitken,1995-09-23,British
850,tsunoda,Yuki,Tsunoda,2000-05-11,Japanese
851,mazepin,Nikita,Mazepin,1999-03-02,Russian


,driverRef_x,driverRef_y,connection
0,hamilton,kovalainen,constructor
1,hamilton,alonso,constructor
2,hamilton,button,constructor
3,hamilton,rosberg,constructor
4,hamilton,bottas,constructor
...,...,...,...
4984,wehrlein,ocon,constructor
4985,stroll,sirotkin,constructor
4986,russell,latifi,constructor
4987,latifi,aitken,constructor


In [29]:
edges[edges.driverRef_x == 'norris']

,driverRef_x,driverRef_y,connection


In [30]:
edges[edges.driverRef_y == 'norris']

,driverRef_x,driverRef_y,connection
4962,ricciardo,norris,constructor
4981,sainz,norris,constructor
